In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)


Toggle code

# Head Check Analysis in Python - Concept and Validation
*Eric G. Suchanek, Ph.D. v.1, 3/9/19*

The following Python code block reads .fit files from the Axiom camera at Chews Ridge as part of Dr. Shane's 'headcheck'
procedure, and extracts a number of header fields:
* obs-date
* exposure time 
* airmass
* time start
* object
* RA
* DEC
* Detector
* Filter
* Darktime

The only fields necessary for the relevant calculations are:
* Observatory location
* observation date
* exposure duration
* object Right Ascension
* Object Declination
* Observatory location. 

The other fields are retrieved as part of code validation and to provide a 
The program computes the time mid, time end, HA, Airmass, 
in UTC and LST by using the coordinates of Chews Ridge, This is used to compute the object's
HA, altitude and airmass. These results are displayed in tabular form. The raw image and a stretched
version are shown side by side. The stretched version is normalized over the image intensities
code automatically adjusts the timezone offset by checking to see if DST is in effect.

In [2]:
#
# Analyze fit files from the Axiom at Chews Ridge as part of Whitney's 'headcheck' procedure.
# This code reads .fits frames from a specified path,extracts the obs-date, exposure time, 
# airmass, time start, object, RA, DEC.
# The program computes the time mid, time end, HA, Airmass, 
# in UTC and LST by using the coordinates of Chews Ridge, This is used to compute the object's
# HA, altitude and airmass. These results are displayed in tabular form. The raw image and a stretched
# version are shown side by side. The stretched version is normalized over the image intensities
# code automatically adjusts the timezone offset by checking to see if DST is in effect.
# 
# -egs- 3/9/19 

from glob import glob
import os
import time
import datetime

import astropy.units as u
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, Latitude, Longitude
from astropy.io import fits
from astropy.time import Time

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from astropy.visualization import (SqrtStretch,ImageNormalize,PercentileInterval)

#
# DST calculation for a given date
# expects date to be .unix format

def is_dst(date):
    return bool(time.localtime(date).tm_isdst)

# Set up our observation location.
# OOS coordinates taken from Google Earth
sitelong = '-121:34:1'
sitelat = '36:18:20'
height = 1516*u.m
site_location = EarthLocation(sitelong, sitelat, height=height)

# Base constant UTC offset for US/Pacific. This can change to -7 hour when DST is in effect.
# the local var utcoffset reflects the ACTUAL utcoffset for the given date, taking DST into account
UTC_offset = -8*u.hour

# process only the light frames. this command produces a list of files matching the string L*.fits
path = './PHOT030727/C/'
files = glob(path + 'L*.fits')

start_time = time.time()
count = 0

# Loop over all files
for fits_image_file in files:
    data, hdr = fits.getdata(fits_image_file, header=True, ext=0)
    # collect parameters from the hdr
    date_obs = hdr['date-obs']
    exptime = hdr['exptime']
    obj_name = hdr['object']
    airmass = hdr['airmass']
    obj_ra = hdr['ra']
    obj_dec = hdr['dec']
    obj_detector = hdr['detector']
    obj_filter = hdr['filter']
    obj_darktime = hdr['darktime']
    exposure_time = exptime*u.second
    exp2 = 5*u.second
    
    exp3 = exposure_time = exp2
    print(exp3)
    
    # convert the strings for ra and dec into actual angle objects for later use
    ra = Longitude(obj_ra, unit=u.hourangle)
    dec = Latitude(obj_dec, unit=u.deg)

    # Do the time calculations - based on file date-obs from the hdr
  
    UTC_time_observation = Time(date_obs,format='fits',location=site_location)
    UTC_mid_exposure_time = UTC_time_observation + exposure_time / 2
    UTC_end_exposure_time = UTC_time_observation + exposure_time
    
    # check if dst is in effect
    if is_dst(UTC_time_observation.unix):
        utcoffset = UTC_offset + 1*u.hour
        dst_string = 'PDT'
    else:
        dst_string = 'PST'

    # Perform the time calculations on the beginning, mid and end times       
    local_time_observation = Time(UTC_time_observation + utcoffset, location=site_location)
    local_time_observation_mid = local_time_observation + exposure_time / 2
    local_time_observation_end = local_time_observation + exposure_time
    
    # Compute sidereal times for the observation
    sidereal_time_type = 'apparent' # or 'mean'
    sidereal_time_local_start = local_time_observation.sidereal_time(sidereal_time_type) 
    sidereal_time_local_mid = local_time_observation_mid.sidereal_time(sidereal_time_type) 
    sidereal_time_local_end = local_time_observation_end.sidereal_time(sidereal_time_type) 

    sidereal_time_UTC_start = UTC_time_observation.sidereal_time(sidereal_time_type) 
    sidereal_time_UTC_mid = UTC_mid_exposure_time.sidereal_time(sidereal_time_type) 
    sidereal_time_UTC_end = UTC_end_exposure_time.sidereal_time(sidereal_time_type) 

    # object's AltAz computed from RA,Dec and site location with time as input:  UTC
    object_Sky = SkyCoord(ra,dec,location=site_location)
    
    object_observed_camera_mid = SkyCoord(ra,dec,obstime=UTC_mid_exposure_time)
    object_altaz_camera_mid = object_Sky.transform_to(AltAz(obstime=(UTC_mid_exposure_time),
                                                            location=site_location))
                                                            
    computed_airmass_mid = object_altaz_camera_mid.secz
    hourangle_mid = sidereal_time_local_mid - object_observed_camera_mid.ra
    
    print('File:      ', fits_image_file)
    print('Object:    ', obj_name)
    print('Exposure:  ', exposure_time)
    print("Local Time: {} {}".format(local_time_observation.iso, dst_string))
    
    norm = ImageNormalize(data, interval=PercentileInterval(.1))
    fig = plt.figure(figsize=(8,3),dpi=120)
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.imshow(data, origin='lower')
    ax2 = fig.add_subplot(1,2,2)
    im2 = ax2.imshow(data, origin='lower',norm=norm)
    fig.colorbar(im2)
    plt.show()
    
    print('RA:                        ', ra)
    print('Dec:                       ', dec)
    print('HA mid:                    ', hourangle_mid)
    print("Altitude (mid exp):         {0.alt:.6}".format(object_altaz_camera_mid))
    print('Exposure:                  ', exposure_time)
    print('Airmass (hdr):             ', airmass)
    print('Airmass mid,(computed):     {0:.4}'.format(computed_airmass_mid))
    print('JD UTC start exp, (hdr):   ', UTC_time_observation.jd)
    print('JD UTC mid exp:            ', UTC_mid_exposure_time.jd)
    print('JD UTC end exp:            ', UTC_end_exposure_time.jd)
    print('UTC start exp:             ', UTC_time_observation.iso)
    print('UTC mid exp:               ', UTC_mid_exposure_time.iso)
    print('UTC end exp:               ', UTC_end_exposure_time.iso)
    print('Local Time start exp:      ', local_time_observation.iso)
    print('Local Time mid exp:        ', local_time_observation_mid.iso)
    print('Local Time end exp:        ', local_time_observation_end.iso)
    print('LST start:                 ', sidereal_time_local_start)
    print('LST mid:                   ', sidereal_time_local_mid)
    print('LST end:                   ', sidereal_time_local_end)
    print('GST start:                 ', sidereal_time_UTC_start)
    print('GST mid:                   ', sidereal_time_UTC_mid)
    print('GST end:                   ', sidereal_time_UTC_end)
    print('---------------------------------------------------')
    count += 1
    # fits.writeto(fits_image_file, data, hdr, overwrite=True)
end_time = time.time()
elapsed_time = (end_time - start_time)*u.s

print('Processed', count, 'files in:', elapsed_time)

5.0 s


File:       ./PHOT030727/C/L110_6.fits
Object:     Landolt 110a
Exposure:   5.0 s
Local Time: 2003-07-28 03:10:57.760 PDT


<Figure size 960x360 with 3 Axes>

RA:                         18h43m09.532s
Dec:                        -0d29m26.231s
HA mid:                     -3h16m54.63s
Altitude (mid exp):         26.4518 deg
Exposure:                   5.0 s
Airmass (hdr):              2.203
Airmass mid,(computed):     2.245
JD UTC start exp, (hdr):    2452848.9242796297
JD UTC mid exp:             2452848.924308565
JD UTC end exp:             2452848.9243375
UTC start exp:              2003-07-28 10:10:57.760
UTC mid exp:                2003-07-28 10:11:00.260
UTC end exp:                2003-07-28 10:11:02.760
Local Time start exp:       2003-07-28 03:10:57.760
Local Time mid exp:         2003-07-28 03:11:00.260
Local Time end exp:         2003-07-28 03:11:02.760
LST start:                  15h26m12.3951s
LST mid:                    15h26m14.902s
LST end:                    15h26m17.4088s
GST start:                  22h27m21.3924s
GST mid:                    22h27m23.8993s
GST end:                    22h27m26.4061s
---------------------------

<Figure size 960x360 with 3 Axes>

...

RA:                         18h43m10.224s
Dec:                        -0d29m23.82s
HA mid:                     -7h25m54.4837s
Altitude (mid exp):         52.7538 deg
Exposure:                   5.0 s
Airmass (hdr):              1.239
Airmass mid,(computed):     1.256
JD UTC start exp, (hdr):    2452848.7518447917
JD UTC mid exp:             2452848.751873727
JD UTC end exp:             2452848.751902662
UTC start exp:              2003-07-28 06:02:39.390
UTC mid exp:                2003-07-28 06:02:41.890
UTC end exp:                2003-07-28 06:02:44.390
Local Time start exp:       2003-07-27 23:02:39.390
Local Time mid exp:         2003-07-27 23:02:41.890
Local Time end exp:         2003-07-27 23:02:44.390
LST start:                  11h17m13.2335s
LST mid:                    11h17m15.7403s
LST end:                    11h17m18.2472s
GST start:                  18h18m22.2309s
GST mid:                    18h18m24.7377s
GST end:                    18h18m27.2446s
-----------------------

<Figure size 960x360 with 3 Axes>

RA:                         15h38m57.013s
Dec:                        -0d17m46.971s
HA mid:                     -6h00m06.7276s
Altitude (mid exp):         50.7544 deg
Exposure:                   5.0 s
Airmass (hdr):              1.293
Airmass mid,(computed):     1.291
JD UTC start exp, (hdr):    2452848.68368125
JD UTC mid exp:             2452848.6837101853
JD UTC end exp:             2452848.68373912
UTC start exp:              2003-07-28 04:24:30.060
UTC mid exp:                2003-07-28 04:24:32.560
UTC end exp:                2003-07-28 04:24:35.060
Local Time start exp:       2003-07-27 21:24:30.060
Local Time mid exp:         2003-07-27 21:24:32.560
Local Time end exp:         2003-07-27 21:24:35.060
LST start:                  9h38m47.7785s
LST mid:                    9h38m50.2854s
LST end:                    9h38m52.7922s
GST start:                  16h39m56.776s
GST mid:                    16h39m59.2828s
GST end:                    16h40m01.7896s
----------------------------

<Figure size 960x360 with 3 Axes>

RA:                         15h38m56.797s
Dec:                        -0d17m42.259s
HA mid:                     -5h22m14.4176s
Altitude (mid exp):         46.8087 deg
Exposure:                   5.0 s
Airmass (hdr):              1.373
Airmass mid,(computed):     1.372
JD UTC start exp, (hdr):    2452848.7099068286
JD UTC mid exp:             2452848.709935764
JD UTC end exp:             2452848.709964699
UTC start exp:              2003-07-28 05:02:15.950
UTC mid exp:                2003-07-28 05:02:18.450
UTC end exp:                2003-07-28 05:02:20.950
Local Time start exp:       2003-07-27 22:02:15.950
Local Time mid exp:         2003-07-27 22:02:18.450
Local Time end exp:         2003-07-27 22:02:20.950
LST start:                  10h16m39.8725s
LST mid:                    10h16m42.3794s
LST end:                    10h16m44.8862s
GST start:                  17h17m48.8699s
GST mid:                    17h17m51.3768s
GST end:                    17h17m53.8836s
----------------------